In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns;
sns.set(color_codes=True);

In [ ]:
transfusion=pd.read_csv("transfusion.data")

In [ ]:
transfusion.head()

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0


In [ ]:
transfusion.to_csv("transfusion.csv",index=False)

In [ ]:
transfusion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 5 columns):
 #   Column                                      Non-Null Count  Dtype
---  ------                                      --------------  -----
 0   Recency (months)                            748 non-null    int64
 1   Frequency (times)                           748 non-null    int64
 2   Monetary (c.c. blood)                       748 non-null    int64
 3   Time (months)                               748 non-null    int64
 4   whether he/she donated blood in March 2007  748 non-null    int64
dtypes: int64(5)
memory usage: 29.3 KB


In [ ]:
transfusion.rename({"whether he/she donated blood in March 2007":"target"},axis=1,inplace=True)

In [ ]:
transfusion.head()

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),target
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0


#target incidence

In [ ]:
transfusion.target.value_counts(normalize=True)

0    0.762032
1    0.237968
Name: target, dtype: float64

In [ ]:
target_column=transfusion.target
feature_column=transfusion.drop("target",axis=1)

In [ ]:
feature_column.head()

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months)
0,2,50,12500,98
1,0,13,3250,28
2,1,16,4000,35
3,2,20,5000,45
4,1,24,6000,77


In [ ]:
target_column.head()

0    1
1    1
2    1
3    1
4    0
Name: target, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(feature_column,target_column,test_size=0.25,random_state=42,stratify=transfusion["target"])

In [ ]:
X_train.head(2)

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months)
334,16,2,500,16
99,5,7,1750,26


In [ ]:
!pip install tpot

     |████████████████████████████████| 92kB 3.8MB/s 
     |████████████████████████████████| 163kB 8.4MB/s 
  Created wheel for stopit: filename=stopit-1.1.2-cp36-none-any.whl size=11956 sha256=9298bfaf367d5f8270ceb9682998b09aa3a0f3e8c24993e2a3a40c2a69bbd693
  Stored in directory: /root/.cache/pip/wheels/3c/85/2b/2580190404636bfc63e8de3dff629c03bb795021e1983a6cc7
Successfully built stopit


In [ ]:
from tpot import TPOTClassifier
from sklearn.metrics import roc_auc_score

In [ ]:
tpot=TPOTClassifier(generations=5,population_size=50,verbosity=2,scoring="roc_auc",random_state=42,config_dict="TPOT light")

In [ ]:
tpot.fit(X_train,y_train)


Generation 1 - Current best internal CV score: 0.7422459184429089

Generation 2 - Current best internal CV score: 0.7422459184429089

Generation 3 - Current best internal CV score: 0.7463508599623511

Generation 4 - Current best internal CV score: 0.7464002790541777

Generation 5 - Current best internal CV score: 0.7464002790541777

Best pipeline: LogisticRegression(MultinomialNB(input_matrix, alpha=1.0, fit_prior=True), C=25.0, dual=False, penalty=l2)


TPOTClassifier(config_dict='TPOT light', crossover_rate=0.1, cv=5,
               disable_update_check=False, early_stop=None, generations=5,
               log_file=None, max_eval_time_mins=5, max_time_mins=None,
               memory=None, mutation_rate=0.9, n_jobs=1, offspring_size=None,
               periodic_checkpoint_folder=None, population_size=50,
               random_state=42, scoring='roc_auc', subsample=1.0, template=None,
               use_dask=False, verbosity=2, warm_start=False)

In [ ]:
# tpot.predict_proba(X_test)

In [ ]:
tpot.fitted_pipeline_.steps

[('stackingestimator',
  StackingEstimator(estimator=MultinomialNB(alpha=1.0, class_prior=None,
                                            fit_prior=True))),
 ('logisticregression',
  LogisticRegression(C=25.0, class_weight=None, dual=False, fit_intercept=True,
                     intercept_scaling=1, l1_ratio=None, max_iter=100,
                     multi_class='auto', n_jobs=None, penalty='l2',
                     random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                     warm_start=False))]

In [ ]:
# tpot_pred_y=tpot.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
# tpot_auc_score1=roc_auc_score(y_test,tpot_pred_y)
# tpot_auc_score

0.6563998800959233

In [ ]:
tpot_auc_score=roc_auc_score(y_test,tpot.predict_proba(X_test)[:,1])
tpot_auc_score

0.790368722186904

In [ ]:
print(f'\nAUC score: {np.round(tpot_auc_score,4)}')
print('\nBest pipeline steps:', end='\n')
for idx, (name,transform) in enumerate(tpot.fitted_pipeline_.steps):
    print(f"{idx}...{(name,transform)}")


AUC score: 0.7904

Best pipeline steps:
0...('stackingestimator', StackingEstimator(estimator=MultinomialNB(alpha=1.0, class_prior=None,
                                          fit_prior=True)))
1...('logisticregression', LogisticRegression(C=25.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False))


In [ ]:
print(f"{np.round((X_train.var()),3)}")

Recency (months)              66.929
Frequency (times)             33.830
Monetary (c.c. blood)    2114363.700
Time (months)                611.147
dtype: float64


In [ ]:
X_train.head()

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months)
334,16,2,500,16
99,5,7,1750,26
116,2,7,1750,46
661,16,2,500,16
154,2,1,250,2


In [ ]:
np.log(X_train["Monetary (c.c. blood)"])

334    6.214608
99     7.467371
116    7.467371
661    6.214608
154    5.521461
         ...   
443    6.620073
683    6.620073
16     8.160518
437    6.620073
308    6.907755
Name: Monetary (c.c. blood), Length: 561, dtype: float64

In [ ]:
X_train_normed,X_test_normed=X_train.copy(),X_test.copy()

for df_ in [X_train_normed,X_test_normed]:
    df_["monetary_log"]=np.log(df_["Monetary (c.c. blood)"])
    df_.drop("Monetary (c.c. blood)",axis=1,inplace=True)

In [ ]:
print(f"{np.round((X_train_normed.var()),3)}")

Recency (months)      66.929
Frequency (times)     33.830
Time (months)        611.147
monetary_log           0.837
dtype: float64


In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logreg=LogisticRegression(random_state=42,solver='liblinear')

In [ ]:
logreg.fit(X_train_normed,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
logreg_auc_score = roc_auc_score(y_test,logreg.predict_proba(X_test_normed)[:,1])
print(f'\nAUC score: {np.round(logreg_auc_score,4)}')


AUC score: 0.789


In [ ]:
from operator import itemgetter

In [ ]:
sorted(
    [('tpot', tpot_auc_score), ('logreg', logreg_auc_score)],
    key=itemgetter(1),
    reverse=True
)

[('tpot', 0.790368722186904), ('logreg', 0.7890178003814368)]